<a href="https://colab.research.google.com/github/PKvasnick/PeakFit/blob/master/PeakFit_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fitování píků

Nacházíte se v Google Colab notebooku, který vám pomůže vybrat a nafitovat píky v spektru. Notebook sídlí v GitHub repozitáři na adrese `https://github.com/PKvasnick/PeakFit/blob/master/PeakFit_dev.ipynb`. Z repozitáře ho otevřete kliknutím na modré tlačítko `Open in Colab`.

**Důležité**  Než začnete s notebookem cokoli dělat, vytvořte si vlastní kopii - z menu `File` vyberte jednu z možností `Save a Copy to...`. Tím nejen získáte lookálni kopii notebooku, kterou si případně můžete modifikovat dle libosti, ale také vytvoříte nové cloudové Pythonovské prostředí pro spouštění notebooku - tedy odlišné od těch, v nichž se práve pokouší spustit notebook vaši spolužáci. 

Jednotlivými kroky procházíte klikaním na trojúhelníčky ▷ v černém poli na levém okraji příslušných buněk. Pokud se spletete, 

* jestli chcete zopakovat poslední krok, klikněte na symbol pro odchod pod šipkou (změní se na x, když na něj najedete myší). Tím se vymaže předchozí výstup buňky. Pak znova klikněte na trojúhelníček pro opakované spuštění kódu tohoto kroku.
* prostě celý notebook znova načtěte, nebo zvolte z menu `Runtime -> Restart Runtime` a začněte znovu. 

Pokud vás zajímá kód, kterým se realizují jednotlivé kroky, klikněte pravým tlačítkem do šedého pásu kolem trojúhelníčku, a zvolte `Form -> Show code`.

Jakékoli stížnosti a připomínky prosím na `peter.kvasnicka@mff.cuni.cz`



## 1. Načtení spektra vybraného vzorku a spektra pozadí
V tomto kroku načtete ze soubovorého systému vašeho počítače datové soubory pro požadovaný vzorek a pro pozadí,`Vzorek-X_YYYs.dat` a `Vzorek-Ref_YYYs.dat`.

Klikněte na trojúhelníček ▷ v černém poli v levém horném rohu u následujících dvou buněk, a když ikonka ožije, klikněte na `Prohledávat` a vyberte požadovaný soubor. 

In [ ]:
#@title <-- Načtěte z počítače soubor pro požadovaný vzorek

import sys
from io import StringIO
from google.colab import files
import numpy as np
import pandas as pd

def read_spectrum():
  '''
  Načte datový soubor ze souborového systému do Pandas
  dataframu a z názvu dekóduje označení dat a čas měření.
  Název souboru se očekává ve tvaru Vzorek-XXX_YYYs.txt.
  '''
  uploaded = files.upload() # Widget, vrací dict (název, obsah)
  filename = list(uploaded.keys())[0]

  # Parse filename
  print('Reading data from: {}'.format(filename))
  sample_id = filename[(filename.find('-')+1):filename.find('_')]
  print('Sample id        : {}'.format(sample_id))
  acc_time_str = filename[(filename.find('_')+1):filename.find('s.txt')]
  acc_time = int(acc_time_str)
  print('Acc. time        : {} sec'.format(acc_time))
  df = pd.read_table(filename, sep = " ")
  print('Read {} rows.'.format(len(df)))
  return sample_id, acc_time, df

sample_id, sample_time, df = read_spectrum()

In [ ]:
#@title <-- Načtěte z počítače soubor pro pozadí

bg_id, bg_time, bg_df = read_spectrum()

# Kontrola
if bg_id != 'Ref':
  print('VAROVÁNÍ: Načtené spektrum není spektrum pozadí!')
  print('Pokud jste se spletli, můžete tento krok zopakovat.')
#





## Sjednocení a zobrazení dat

Pozadí je nutno přeškálovat podle doby měření, aby ho bylo možno odečíst od spektra vzorku. Pak si zobrazíme výslednou tabulku, abychom viděli, že se data načetla rozumně. 

In [ ]:
#@title <-- Sjednocení a zobrazení dat

# Sjednotíme data - careful with data types
# FIXME: Normalizing with a Bulgarian constant
amplif = sample_time / bg_time
df['Bg_counts'] = (bg_df.Counts * amplif + 0.5).astype(int)
df['Net_counts'] = df.Counts - df.Bg_counts

# Initial range for plot
xmin = np.min(df.Energy_keV.values)
xmax = np.max(df.Energy_keV.values)
xm = 0.5 * (xmin + xmax)
d = 0.5 * (xmax - xmin)
x0 = xm - 0.5 * d
x1 = xm + 0.5 * d

df

## 3. Vykreslení spektra vzorku a blanku

V tomto kroku si nakreslíme spektra vzroku, blanku a jejich rozdílu, abychom si mohli vybrat oblast, kde chceme nafitovat pík, a zvolit typ pozadí. 

In [ ]:
#@title <-- V zobrazeném spektru vyberte oblast kolem píku (ROI) a zvolte tvar pozadí

from plotly import graph_objects as go
from IPython.display import display, clear_output
import ipywidgets as widgets

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df.Energy_keV, y=df.Bg_counts, name='Blank', mode = 'lines', line_width = 0.7)
    ).add_trace(
    go.Scatter(x=df.Energy_keV, y=df.Counts, name='Sample', mode = 'lines', line_width = 0.7)
    ).add_trace(
    go.Scatter(x=df.Energy_keV, y=df.Net_counts, name='Net sample')
    )

# Set title
fig.update_layout(
    title_text="Spektrum vzorku a pozadí"
)

# -------------------------------------------------------
# Oblast ROI
# -------------------------------------------------------

fig.update_layout(
    shapes = [
      {
        "x0": x0,
        "x1": x1,
        "y0": 0,
        "y1": 1,
        "type": "rect",
        "xref": "x",
        "yref": "paper",
        "opacity": 0.2,
        "fillcolor": "#d3d3d3"
      }]
)

# -------------------------------------------------------
# ROI slider
# -------------------------------------------------------

rs = widgets.FloatRangeSlider(
    value=[x0, x1],
    min=xmin,
    max=xmax,
    step=0.05,
    description='ROI',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout = widgets.Layout(width = '94%')
)

def on_range_update(slider):
  fig.update_shapes(dict(x0 = rs.value[0], x1 = rs.value[1]))
  clear_output()
  fig.show()
  display(rs)
  display(bg_buttons)

rs.observe(on_range_update)

# -----------------------------------------------------------------------
# Tvar pozadí
# -----------------------------------------------------------------------
bg_labels = ['Konstantní pozadí', 
             'Lineární pozadí', 
             'Kvadratické pozadí']

output_radio_selected = widgets.Text('Kvadratické pozadí')

bg_buttons = widgets.RadioButtons(
    options=bg_labels,
    value=bg_labels[2],
    description='Typ pozadí:',
    layout={'width': 'max-content'}, 
    disabled=False
)

def bg_buttons_observer(sender):
    output_radio_selected.value = bg_buttons.value

bg_buttons.observe(bg_buttons_observer, names=['value'])
# -------------------------------------------------------
# Vysypeme na displej
# -------------------------------------------------------

fig.show()
display(rs)
display(bg_buttons)

Několik rad:
* Může se stát, že vaše spektrum je široké a úzký pík se vám pomocí slajdru špatně vybírá. Nic nezkazíte, když ze zdrojových dat odmažete nezajímavou část spektra a začnete znovu, z užšího spektra se vám bude vybírat pohodlněji. 
* Vybírejte pík vždy i s malým okolím. Pozadí se fituje kvadratickým polynomem, a fit bude hezčí, když se bude mít čeho chytit. 
* Pokud fitujete hodně asymetrický pík, zkuste fit s lineárním nebo konstantním pozadím.
* Zvolte konstantní pozadí, fitujete-li osamělý pík.

## 4. Fitování píku

Po spuštění se program pokusí nalézt optimální kombinaci gaussiánu a polynomu zvoleného stupně, která co nejlépe vystihujte spektrum ve zvolené oblasti. Nemusí se vám to povest napoprvé, v případě neúspěchu klidně začněte od 3. kroku.

In [ ]:
#@title <-- Nafitujte vybraný pík

from scipy.stats import norm
from scipy.optimize import curve_fit
from math import sqrt, exp, log

# Subset df acc to ROI
low, high = rs.value

df_fit = df[np.logical_and(df.Energy_keV > low, df.Energy_keV < high)].copy()
df_fit['Errors'] = np.sqrt(df_fit.Counts)

# Dynamic definition of polynomial part
bg_degree = 0
while bg_buttons.value != bg_labels[bg_degree]:
  bg_degree += 1

init_polyvals = [0] * (bg_degree + 1)

def bg_poly(x, a):
  p = np.zeros_like(x)
  for i in range(len(a)):
    p = p*x + a[i]
  return p

def gauss(x, N0, mean, sigma):
  return N0 * norm.pdf(x, mean, sigma)

def gauss_and_bg(x, N0, mean, sigma, *polyvals):
  return bg_poly(x, polyvals) + gauss(x, N0, mean, sigma)
  
popt, pcov = curve_fit(
    gauss_and_bg, 
    df_fit.Energy_keV, 
    df_fit.Net_counts, 
    [1000,0.5*(high+low), 0.5*(high-low), *init_polyvals] 
)

# Peak stats:
N0 = np.sum(df_fit.Net_counts)
dE = np.mean(np.diff(df.Energy_keV))
sig2fwhm = 2*sqrt(2*log(2))
peak_stats = pd.DataFrame(dict(
    Parameter=['left_edge', 'right_edge','centroid', 'FWHM', 'N_total', 'N_peak', 'N_bg', 'Acc. time', 'Rate'],
    Unit=['keV', 'keV', 'keV', 'keV', '', '', '', 's', 'Hz'],
    Value=[low, high, popt[1], sig2fwhm*popt[2], N0, popt[0]/dE, N0 - popt[0]/dE, sample_time, popt[0]/dE/sample_time],
    Error=[None,None,sqrt(pcov[1,1]),sig2fwhm*sqrt(pcov[2,2]),None,sqrt(pcov[0,0])/dE,sqrt(pcov[0,0])/dE, 0, sqrt(pcov[0,0])/dE/sample_time]
))

df_fit['Fit'] = df_fit.apply(lambda row: gauss_and_bg(row.Energy_keV, *popt), axis = 1)
df_fit['Bg'] = df_fit.apply(lambda row: bg_poly(row.Energy_keV, popt[3:]), axis = 1)

fig.add_trace(
    go.Scatter(x=df_fit.Energy_keV, y=df_fit.Fit)
)
fig.add_trace(
    go.Scatter(x=df_fit.Energy_keV, y=df_fit.Bg)
)
fig.show()

print('PEAK FIT SUMMARY:')
pd.options.display.float_format = '{:.2f}'.format
peak_stats

Může se stát, že fit neskonverguje nebo bude špatný. V takovém případě 
* zkuste mírně změnit výběr oblasti fitování ve 3. kroku
* zkuste jiný tvar pozadí

Postup můžete opakovat pro další píky i bez resetování notebooku, od 3. kroku. Pokud se chcete zbavit předchozích fitů, vymažte výstup ve 3. kroku kliknutím na symbol pod spouštěcí šipkou.

**Důležité** Tabulka parametrů píku se dalším fitem přepíše, takže si ji zkopírujte, než přejdete k novému spektru.

## Poznámky

1. Pokud si chcete udělat pořádek v souborech, které jste už načetli, klikněte na symbol 📁 nahoře nalevo. 
2. Změny v původní notebooku, otevřeném z GitHub repozitáře, vám systém nedovolí uložit. Udělejte si vlastní kopii, nejsnadněji ho skopírujete na Google Drive.